Notebook demonstrating document querying and answer generation using Llamaindex, Pinecone and openai.

# Imports

In [ ]:
import os
import openai
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
# set api keys
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_KEY = os.environ["PINECONE_KEY"]

In [ ]:
openai.api_key = OPENAI_API_KEY

In [ ]:
pinecone_client = Pinecone(api_key=PINECONE_KEY)

In [ ]:
index_name = "rag-demo"

pinecone_index = pinecone_client.Index(index_name)
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=OpenAIEmbedding())
retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=5)

In [ ]:
query = "List down some text-to-image models"
candidates = retriever.retrieve(query) # retrieve the close candidates

In [ ]:
context = "\n\n\n".join([x.metadata["window"] for x in candidates])

In [ ]:
for candidate in candidates:
    print(f"{candidate.score:.2f}\t{candidate.text}")
    print()

In [ ]:
prompt = "Answer the question: {question}. \n\n Use only the given context to answer the question. You can format the answer in bullet points. The context is: \n {context}"
prompt = prompt.format(question=query, context=context)

In [ ]:
print(prompt) # print the prompt

In [ ]:
response = openai.chat.completions.create(model="gpt-3.5-turbo", temperature=0.3, messages=[{"role": "user", "content": prompt}])

In [ ]:
answer = response.choices[0].message.content

In [ ]:
print(answer)

In [ ]:
# combine retrival and answer generation in a single function
def get_answer(query):
    candidates = retriever.retrieve(query)
    context = "\n\n\n".join([x.metadata["window"] for x in candidates]) # using window for context
    prompt = "Answer the question: {question}. \n\n Use only the given context to answer the question. You can format the answer in bullet points. The context is: \n {context}"
    prompt = prompt.format(question=query, context=context)
    
    response = openai.chat.completions.create(model="gpt-3.5-turbo", temperature=0.3, messages=[{"role": "user", "content": prompt}])
    answer = response.choices[0].message.content
    
    return answer

In [ ]:
print(get_answer("Who are the authors of the report?"))

In [ ]:
print(get_answer("Which is the most in-demand field of AI?"))

In [ ]:
print(get_answer("What is the global investment made in AI In 2022?"))